### Creación de un cargador de datos para NLP

Como ingeniero de IA en un proyecto avanzado de traducción de idiomas, tu objetivo es superar las barreras lingüísticas enfrentando los desafíos que presentan los matices y contextos culturales. El éxito del modelo depende en gran medida de grandes corpus bilingües utilizados para entrenar los modelos.

En PyTorch, el **cargador de datos** es esencial para manejar estos datos, especialmente por la variabilidad en la longitud de las secuencias en tareas de procesamiento de lenguaje natural (NLP). Este componente agrupa eficientemente secuencias de diferente tamaño, lo que permite un entrenamiento más eficaz aprovechando la computación paralela en GPU.

También mezcla aleatoriamente los datos para evitar que el modelo memorice el orden de entrada, promoviendo una mejor generalización, algo crucial en NLP. Además, facilita la integración de pasos de preprocesamiento como tokenización, padding y conversión numérica, transformando el texto en bruto en datos listos para el modelo.


### Configuraciones
#### Instalando librerías requeridas


In [ ]:
import sys
print(sys.executable)


#### Importando librerías requeridas


In [ ]:
import torchtext
print(torchtext.__version__)

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchdata.datapipes.iter import IterableWrapper, Mapper
import torchtext

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import random

#### **Conjunto de datos**

Un conjunto de datos en **PyTorch** es un objeto que representa una colección de muestras de datos.
Cada muestra de datos típicamente consta de una o más características de entrada y sus correspondientes etiquetas objetivo.  Cada muestra puede ser una oración o un par de oraciones en tareas de traducción.  Es responsable de almacenar y, en algunos casos, transformar los datos brutos en un formato utilizable.

#### **Cargador de datos**

Un cargador de datos en **PyTorch** es responsable de cargar y agrupar en lotes los datos de un conjunto de datos de manera eficiente. Abstrae el proceso de iterar sobre un conjunto de datos, mezclarlos y dividirlos en lotes para el entrenamiento. En aplicaciones de NLP, el cargador de datos se utiliza para procesar y transformar tus datos textuales, en lugar de simplemente utilizar el conjunto de datos.

Los cargadores de datos tienen varios parámetros clave, incluyendo el conjunto de datos desde el cual se cargan, el tamaño del lote (que determina cuántas muestras se incluyen por lote), si se mezclan los datos (shuffle, para cada época) y la aplicación de transformaciones adicionales a través de funciones de colación (collate functions). 

Los cargadores de datos también proporcionan una interfaz de iterador, lo que facilita iterar sobre los lotes de datos durante el entrenamiento.

#### '**¿Qué es un iterador?**'

Un iterador es un objeto sobre el cual se puede iterar. Contiene elementos que se pueden recorrer y típicamente incluye dos métodos, `__iter__()` y `__next__()`. Cuando no hay más elementos para iterar, lanza una excepción **`StopIteration`**.

Los iteradores se utilizan comúnmente para recorrer grandes conjuntos de datos sin cargar todos los elementos en memoria simultáneamente, haciendo el proceso más eficiente en cuanto a memoria. En PyTorch, no todos los conjuntos de datos son iteradores, pero todos los cargadores de datos lo son.

En PyTorch, el cargador de datos procesa la información en lotes (batches), cargando y procesando un lote a la vez en memoria de manera eficiente. El tamaño del lote, que se especifica al crear el cargador de datos, determina cuántas muestras se procesan juntas en cada lote.  El propósito del cargador de datos es convertir los datos de entrada y las etiquetas en lotes de tensores con la misma forma para que los modelos de aprendizaje profundo puedan interpretarlos.

El propósito principal de estas herramientas es transformar datos textuales, que en su forma original son cadenas de caracteres, en tensores numéricos. Estos tensores se obtienen mediante procesos de tokenización, conversión de tokens a índices (utilizando un vocabulario) y, finalmente, agrupación en lotes de tamaño fijo mediante técnicas como el padding.


### Ejemplo

In [ ]:
sentences = [
    "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.",
    "Fame's a fickle friend, Harry.",
    "It is our choices, Harry, that show what we truly are, far more than our abilities.",
    "Soon we must all face the choice between what is right and what is easy.",
    "Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.",
    "You are awesome!"
]

# Definir un conjunto de datos personalizado
class CustomDataset(Dataset):
    def __init__(self, sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

# Crear una instancia de tu conjunto de datos personalizado
custom_dataset = CustomDataset(sentences)

# Definir tamaño del lote
batch_size = 2

# Crear un DataLoader
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Iterar a través del DataLoader
for batch in dataloader:
    print(batch)

En este ejemplo el conjunto de datos consta de una lista de oraciones aleatorias, y el objetivo es crear lotes de oraciones para su posterior procesamiento, como el entrenamiento de un modelo de red neuronal.

##### **Creación de un conjunto de datos personalizado**

En muchos casos, los datos disponibles no se ajustan a ningún formato predefinido, por lo que se requiere la creación de un conjunto de datos personalizado. Este proceso implica:

1. **Heredar de la clase `torch.utils.data.Dataset`:**  
   Esto obliga a implementar dos métodos fundamentales:
   - **`__len__`:** Devuelve el número total de muestras disponibles en el conjunto de datos.
   - **`__getitem__`:** Permite acceder a una muestra individual en función de su índice.

2. **Almacenar las muestras y, opcionalmente, aplicar transformaciones:**  
   En el ejemplo proporcionado, el conjunto de datos consiste en una lista de oraciones. Inicialmente, se define una clase simple que retorna la oración en bruto según su índice.

En el ejemplo anterior, se crea un objeto `CustomDataset` que recibe una lista de oraciones. La función `__getitem__` permite extraer cada oración individualmente, y `__len__` asegura que se conozca el número total de muestras. 

Además, puedes especificar un tamaño de lote (`batch_size`), que determina cuántas oraciones se agruparán en cada lote durante la carga de datos.

##### **Uso del dataLoader y el concepto de iterador**

Una vez creado el conjunto de datos personalizado, se utiliza el **DataLoader** de PyTorch para iterar sobre los datos de manera eficiente. Se especifica el tamaño del lote (`batch_size`) y se establece el parámetro `shuffle=True` para mezclar aleatoriamente las muestras antes de formar cada lote. Esto es crucial para evitar sesgos en el entrenamiento del modelo ya que evita que el modelo aprenda patrones basados en el orden de los datos.

En el ejemplo, el **DataLoader** agrupa las oraciones en lotes de dos. Cada iteración del bucle imprime un lote, demostrando que el cargador procesa las muestras en bloques de tamaño fijo. Internamente, el DataLoader utiliza un iterador para recorrer el conjunto de datos, lo que permite trabajar de manera eficiente sin cargar todo el conjunto de datos en memoria de una sola vez.

#### Conversión de oraciones a tensores y creación de vocabulario

Los modelos de aprendizaje profundo requieren que los datos de entrada sean numéricos. Por ello, las oraciones deben transformarse en tensores. Este proceso implica dos pasos esenciales:

1. **Tokenización:**  : La conversión de una oración en una lista de palabras o tokens. Se puede realizar utilizando funciones como `get_tokenizer` de TorchText, que permite elegir distintos métodos de tokenización. Por ejemplo, el tokenizador "basic_english" divide el texto en palabras simples, eliminando signos de puntuación y convirtiendo el texto a minúsculas.

2. **Construcción de un vocabulario:** : Una vez tokenizadas las oraciones, es necesario construir un vocabulario que asigne a cada token un índice único. La función `build_vocab_from_iterator` de TorchText recorre todas las oraciones tokenizadas para generar este mapeo. Este vocabulario es esencial para convertir cada token en un valor numérico que pueda ser interpretado por el modelo.

En este ejemplo de código, veremos la creación de un conjunto de datos personalizado para tareas de procesamiento de lenguaje natural (NLP) utilizando PyTorch. El conjunto de datos consiste en una lista de oraciones, y el objetivo del código es preprocesar estas oraciones, tokenizarlas y convertirlas en tensores de índices de tokens para su uso en modelos de NLP.

In [ ]:
sentences = [
    "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.",
    "Fame's a fickle friend, Harry.",
    "It is our choices, Harry, that show what we truly are, far more than our abilities.",
    "Soon we must all face the choice between what is right and what is easy.",
    "Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.",
    "You are awesome!"
]

# Definir un conjunto de datos personalizado
class CustomDataset(Dataset):
    def __init__(self, sentences, tokenizer, vocab):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.vocab = vocab

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        tokens = self.tokenizer(self.sentences[idx])
        # Convertir tokens a índices de tensor usando el vocabulario
        tensor_indices = [self.vocab[token] for token in tokens]
        return torch.tensor(tensor_indices)

# Tokenizador
tokenizer = get_tokenizer("basic_english")

# Construir el vocabulario
vocab = build_vocab_from_iterator(map(tokenizer, sentences))

# Crear una instancia de tu conjunto de datos personalizado
custom_dataset = CustomDataset(sentences, tokenizer, vocab)

print("Longitud del conjunto de datos personalizado:", len(custom_dataset))
print("Items de muestra:")
for i in range(6):
    sample_item = custom_dataset[i]
    print(f"Item {i + 1}: {sample_item}")

**Desglosaremos el código paso a paso.**

Las oraciones y la clase `CustomDataset` se utilizan de la misma manera que en el fragmento de código dado al inicio. Los cambios realizados en la clase `CustomDataset` son los siguientes:

- `__init__`: El constructor recibe como entrada una lista de oraciones, una función de tokenización y un vocabulario (`vocab`).
- `__len__`: Este método devuelve el número total de muestras en el conjunto de datos.
- `__getitem__`: Este método se encarga de procesar una muestra individual. Tokeniza la oración utilizando el tokenizador proporcionado y luego convierte los tokens en índices de tensores mediante el vocabulario.

La **tokenización** consiste en convertir una oración en una lista de palabras o tokens. Esto se puede realizar utilizando funciones como `get_tokenizer` de TorchText, que permite elegir distintos métodos de tokenización. Por ejemplo, el tokenizador `"basic_english"` divide el texto en palabras simples, eliminando signos de puntuación y convirtiendo el texto a minúsculas.

Una vez tokenizadas las oraciones, es necesario construir un **vocabulario** que asigne a cada token un índice único. La función `build_vocab_from_iterator` de TorchText recorre todas las oraciones tokenizadas para generar este mapeo. Este vocabulario es esencial para convertir cada token en un valor numérico que pueda ser interpretado por el modelo.

Luego, se crea una instancia del conjunto de datos personalizado, pasando las oraciones, el tokenizador y el vocabulario. Finalmente, se imprime la longitud del conjunto de datos y algunos elementos de muestra para ilustrar su estructura y funcionamiento.

Descomenta el siguiente código para aplicar el cargador de datos y observar los resultados:

In [ ]:
"""
# Crear una instancia de tu conjunto de datos personalizado
custom_dataset = CustomDataset(sentences, tokenizer, vocab)

# Definir tamaño del lote
batch_size = 2

# Crear un cargador de datos
#dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Iterar a través del cargador de datos
for batch in dataloader:
    print(batch)
"""

> Resultado: Debes encontrar un error al intentar crear lotes de tensores debido a que estos tienen **longitudes desiguales**. Este error ocurre porque el cargador de datos está utilizando la **función de colación por defecto** (`collate_fn`), la cual requiere que todos los tensores dentro de un lote tengan la **misma longitud**.

Para solucionar este problema, se puede **definir una función de colación personalizada**, donde se establece las reglas para procesar los datos antes de formar los lotes.

Una estrategia común para manejar tensores de diferentes longitudes es aplicar **padding**, es decir, rellenar las secuencias más cortas con un valor especial (como ceros) hasta igualar la longitud de la más larga. 


#### Función de colación personalizada

El DataLoader en PyTorch utiliza una función de colación para combinar múltiples muestras en un solo lote. La función de colación por defecto no puede manejar tensores de longitudes variables, lo que puede generar errores. Por ello, se define una función de colación personalizada que aplica padding a las secuencias.

A continuación se muestra un ejemplo de una función de colación personalizada:

In [ ]:
# Crear una función de colación personalizada
def collate_fn(batch):
    # Rellenar las secuencias dentro del lote para que tengan longitudes iguales
    padded_batch = pad_sequence(batch, batch_first=True, padding_value=0)
    return padded_batch

# Crear un cargador de datos con la función de colación personalizada con batch_first=True,
dataloader = DataLoader(custom_dataset, batch_size=batch_size, collate_fn=collate_fn)

# Iterar a través del cargador de datos
for batch in dataloader: 
    for row in batch:
        for idx in row:
            words = [vocab.get_itos()[idx] for idx in row]
        print(words)

En la celda anterior, al rellenar las secuencias, se establece `batch_first=True`. Cuando se usa `batch_first=True`, la salida tendrá la forma `[batch_size x sequence_length]`, de lo contrario, tendrá la forma `[sequence_length x batch_size]`. 

Algunos modelos aceptan entradas con la forma `[batch_size x sequence_length]` ( modelos de NLP, ya que cada fila representa una muestra y cada columna un token en la secuencia) mientras que otros necesitan que la entrada tenga la forma `[sequence_length x batch_size]` (algunas implementaciones de RNN, donde se espera que la secuencia sea la dimensión principal). 


Al observar el resultado, se observa que  la primera dimensión es el lote.  Por ejemplo, el primer lote es la primera oración: 

```
"['if', 'you', 'want', 'to', 'know', 'what', 'a', 'man', "'", 's', 'like', ',', 'take', 'a', 'good', 'look', 'at', 'how', 'he', 'treats', 'his', 'inferiors', ',', 'not', 'his', 'equals', '.']".
```


In [ ]:
# Iterar a través del cargador de datos con batch_first = TRUE
for batch in dataloader:    
    print(batch)
    print("Longitud de las secuencias en el lote:", batch.shape[1])

El siguiente ejemplo muestra cómo se crea y utiliza una función de colación personalizada con `batch_first=False`:


In [ ]:
# Crear una función de colación personalizada
def collate_fn_bfFALSE(batch):
    # Rellenar las secuencias dentro del lote para que tengan longitudes iguales
    padded_batch = pad_sequence(batch, padding_value=0)
    return padded_batch
    
# Crear un cargador de datos con la función de colación personalizada (batch_first=False)
dataloader_bfFALSE = DataLoader(custom_dataset, batch_size=batch_size, collate_fn=collate_fn_bfFALSE)

# Iterar a través del cargador de datos
for seq in dataloader_bfFALSE:
    for row in seq:
        #print(row)
        words = [vocab.get_itos()[idx] for idx in row]
        print(words)

En este caso, al iterar sobre el DataLoader, se observa que la primera dimensión del tensor corresponde a la secuencia y no al lote. Esto significa que cada fila del tensor representa la posición de un token en todas las oraciones del lote. 

> Este formato es fundamental tenerlo en cuenta para evitar confusiones al diseñar y entrenar modelos que requieran una forma específica de entrada.


### Implementación completa

In [ ]:
# Define un conjunto de datos personalizado
class CustomDataset(Dataset):
    def __init__(self, sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

In [ ]:
custom_dataset = CustomDataset(sentences)

In [ ]:
custom_dataset[0]

In [ ]:
def collate_fn(batch):
    # Tokenizar cada muestra en el lote usando el tokenizador especificado
    tensor_batch = []
    for sample in batch:
        tokens = tokenizer(sample)
        # Convertir los tokens a índices del vocabulario y crear un tensor para cada muestra
        tensor_batch.append(torch.tensor([vocab[token] for token in tokens]))

    # Rellenar las secuencias dentro del lote para que tengan longitudes iguales usando pad_sequence
    # batch_first=True asegura que los tensores tengan la forma (tamaño_lote, longitud_secuencia_maxima)
    padded_batch = pad_sequence(tensor_batch, batch_first=True)
    
    # Devolver el lote rellenado
    return padded_batch

In [ ]:
# Crear un cargador de datos para el conjunto de datos personalizado
dataloader = DataLoader(
    dataset=custom_dataset,   # Conjunto de datos personalizado de PyTorch que contiene tus datos
    batch_size=batch_size,     # Número de muestras en cada mini-lote
    shuffle=True,              # Mezclar los datos al inicio de cada época
    collate_fn=collate_fn      # Función de colación personalizada para procesar los lotes
)

In [ ]:
for batch in dataloader:
    print(batch)
    print("forma de la muestra", len(batch))

Como resultado, se deberían hacer creado con éxito lotes de tensores con longitudes iguales.

#### Ejercicio


Crea un cargador de datos con una función de colación que procese lotes de texto en francés (proporcionado a continuación). Ordena el conjunto de datos según la longitud de las secuencias. Luego, tokeniza, numera y rellena las secuencias. Ordenar las secuencias minimizará la cantidad de tokens `<PAD>` añadidos a las secuencias, lo que mejora el rendimiento del modelo. Prepara los datos en lotes de tamaño 4 e imprímelos.

In [ ]:
corpus = [
    "Ceci est une phrase.",
    "C'est un autre exemple de phrase.",
    "Voici une troisième phrase.",
    "Il fait beau aujourd'hui.",
    "J'aime beaucoup la cuisine française.",
    "Quel est ton plat préféré ?",
    "Je t'adore.",
    "Bon appétit !",
    "Je suis en train d'apprendre le français.",
    "Nous devons partir tôt demain matin.",
    "Je suis heureux.",
    "Le film était vraiment captivant !",
    "Je suis là.",
    "Je ne sais pas.",
    "Je suis fatigué après une longue journée de travail.",
    "Est-ce que tu as des projets pour le week-end ?",
    "Je vais chez le médecin cet après-midi.",
    "La musique adoucit les mœurs.",
    "Je dois acheter du pain et du lait.",
    "Il y a beaucoup de monde dans cette ville.",
    "Merci beaucoup !",
    "Au revoir !",
    "Je suis ravi de vous rencontrer enfin !",
    "Les vacances sont toujours trop courtes.",
    "Je suis en retard.",
    "Félicitations pour ton nouveau travail !",
    "Je suis désolé, je ne peux pas venir à la réunion.",
    "À quelle heure est le prochain train ?",
    "Bonjour !",
    "C'est génial !"
]

In [ ]:
## Tu codigo de respuesta

#### [Opcional] Cargador de datos para la tarea de traducción


Esta sección prepara el terreno para la traducción automática Alemán-Inglés utilizando las librerías torchtext y spaCy. 

Se ajusta las URLs del conjunto de datos Multi30k, configura los tokenizadores para ambos idiomas y establece vocabularios con tokens especiales. 

Esta base es crucial para construir y entrenar modelos de traducción efectivos.

- **Configuración del conjunto de datos y definición del idioma**
  - Se modifican las URLs por defecto del conjunto de datos Multi30k para corregir enlaces rotos.
  - Se definen los idiomas de origen (`de` para alemán) y destino (`en` para inglés).

- **Configuración del tokenizador**
  - Se configuran tokenizadores para ambos idiomas utilizando `spaCy`.

- **Generación de tokens**
  - Se crea una función auxiliar, `yield_tokens`, para generar tokens a partir del conjunto de datos.

- **Símbolos especiales**
  - Se definen los símbolos especiales (por ejemplo, `<unk>`, `<pad>`) y sus índices.

- **Construcción del vocabulario**
  - Se construyen vocabularios para ambos idiomas (origen y destino) a partir de los datos de **entrenamiento** del conjunto de datos Multi30k, convirtiendo tokens en índices únicos (números).

- **Manejo de tokens por defecto**
  - Se establece un índice por defecto (`UNK_IDX`) para los tokens que no se encuentren en el vocabulario.


#### 1. Configuración del conjunto de datos y definición de idiomas

El primer paso consiste en ajustar las URLs del conjunto de datos Multi30k. Dado que las URLs originales estaban rotas, se sobrescriben para apuntar a ubicaciones correctas en un servidor de almacenamiento en la nube. Esto es fundamental para poder acceder a los datos de entrenamiento y validación de manera adecuada:

In [ ]:
# Se modificarán las URLs del conjunto de datos ya que los enlaces al conjunto de datos original están rotos

multi30k.URL["train"] = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/training.tar.gz"
multi30k.URL["valid"] = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/validation.tar.gz"

Posteriormente, se definen dos variables globales para establecer el idioma de origen y el de destino. En este caso, se utiliza el alemán (`de`) como idioma de origen y el inglés (`en`) como idioma de destino:

In [ ]:
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

#### 2. Inicialización del conjunto de datos de traducción

Una vez definidas las URLs y los idiomas, se crea un iterador para el conjunto de datos Multi30k. Esto se hace mediante la función `Multi30k` de torchtext, que permite especificar el conjunto de datos a utilizar (en este caso, 'train') y la pareja de idiomas:

In [ ]:
train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

La variable `train_iter` es un iterador que contiene pares de oraciones, donde la primera corresponde al idioma alemán y la segunda al inglés. Para poder explorar los datos, se convierte el iterador en un objeto iterable:

In [ ]:
data_set = iter(train_iter)

Luego, mediante un ciclo `for`, se imprimen los primeros cinco pares de oraciones (una muestra de entrenamiento) para visualizar el contenido del conjunto de datos. Esto permite confirmar que se están recuperando correctamente los datos de ambos idiomas:

In [ ]:
for n in range(5):
    # Obtener el siguiente par de oraciones de origen y destino del conjunto de datos de entrenamiento
    src, tgt = next(data_set)

    # Imprimir las oraciones de origen (alemán) y destino (inglés)
    print(f"muestra {str(n+1)}")
    print(f"Origen ({SRC_LANGUAGE}): {src}\nDestino ({TGT_LANGUAGE}): {tgt}")

#### 3. Configuración del Tokenizador con spaCy

El siguiente paso es configurar los tokenizadores para ambos idiomas utilizando spaCy, una poderosa biblioteca para el procesamiento del lenguaje natural. Los tokenizadores se encargan de descomponer cada cadena de texto en unidades menores o tokens (palabras, puntuaciones, etc.), lo que permite un procesamiento posterior más fino.



In [ ]:
german, english = next(data_set)
print(f"Origen Alemán ({SRC_LANGUAGE}): {german}\nDestino Inglés ({TGT_LANGUAGE}): { english }")

Primero, se importan las utilidades de tokenización de torchtext:

In [ ]:
from torchtext.data.utils import get_tokenizer

Luego se descargan los modelos de spaCy para inglés y alemán, lo que garantiza que los tokenizadores puedan segmentar correctamente las oraciones según las reglas gramaticales y de puntuación de cada idioma:

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm


Una vez descargados, se crean un diccionario llamado `token_transform` para almacenar los tokenizadores correspondientes a cada idioma:


In [ ]:
# Crear un diccionario para almacenar ambos tokenizadores
token_transform = {}

token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

La línea `token_transform['de'](german)` aplicará el tokenizador configurado para el alemán sobre la cadena de texto almacenada en la variable `german`. De forma similar, `token_transform['en'](english)` tokenizará el texto en inglés.

In [ ]:
token_transform['de'](german)

Lo mismo para el inglés:

In [ ]:
token_transform['en'](english)

#### 4. Símbolos especiales

En un contexto típico de NLP, los tokens `['<unk>', '<pad>', '<bos>', '<eos>']` tienen significados específicos:

1. `<unk>`: Este token representa "desconocido" o "fuera del vocabulario". Se utiliza cuando una palabra en el texto de entrada no se encuentra en el vocabulario o cuando se trata de palabras raras o no vistas durante el entrenamiento. Cuando el modelo se encuentra con una palabra desconocida, la reemplaza con el token `<unk>`.

2. `<pad>`: Este token representa el relleno. En secuencias de datos textuales, como oraciones o documentos, las secuencias pueden tener diferentes longitudes. Para crear lotes de datos con dimensiones uniformes, las secuencias más cortas a menudo se rellenan con este token `<pad>` para igualar la longitud de la secuencia más larga en el lote.

3. `<bos>`: Este token representa el "inicio de secuencia." Se utiliza para indicar el comienzo de una oración o secuencia de tokens. Ayuda al modelo a entender el inicio de una secuencia de texto.

4. `<eos>`: Este token representa el "fin de secuencia." Se utiliza para indicar el final de una oración o secuencia de tokens. Ayuda al modelo a reconocer el final de una secuencia de texto.

Se definen estos símbolos y se asignan índices específicos para cada uno, que se usarán en la construcción del vocabulario y en la transformación de texto a tensores:


In [ ]:
# Definir símbolos especiales e índices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Asegurarse de que los tokens estén en orden de sus índices para insertarlos correctamente en el vocabulario
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

El orden de estos símbolos es crucial para insertarlos correctamente al inicio del vocabulario y para definir el índice por defecto para palabras desconocidas.

#### 5. Construcción del vocabulario a partir del conjunto de datos

El código inicializa un diccionario vocab_transform y luego construye vocabularios para ambos idiomas, alemán (de) e inglés (en), a partir del conjunto de datos ```train_iter``` utilizando la función auxiliar ```yield_tokens```. Estos vocabularios se almacenan en el diccionario vocab_transform. Los vocabularios se construyen con ciertas restricciones, como una frecuencia mínima para los tokens y la inclusión de símbolos especiales al inicio.

Inicializa un diccionario para almacenar los vocabularios de ambos idiomas:

In [ ]:
# Diccionario de reserva para las transformaciones de vocabulario de 'en' y 'de'
vocab_transform = {}

Antes de construir el vocabulario, se define una función auxiliar llamada `yield_tokens` que recibe un iterador de datos y el idioma para el cual se desean extraer tokens. La función mapea la posición de cada idioma en la muestra (por ejemplo, la posición 0 para el alemán y la posición 1 para el inglés) y aplica el tokenizador correspondiente:

In [ ]:
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    # Definir un mapeo para asociar los idiomas de origen y destino con sus respectivas posiciones en las muestras de datos.
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    # Iterar sobre cada muestra de datos en el iterador del conjunto de datos proporcionado
    for data_sample in data_iter:
        # Tokenizar la muestra de datos correspondiente al idioma especificado y devolver los tokens resultantes.
        yield token_transform[language](data_sample[language_index[language]])


Esta función se utiliza para recorrer el conjunto de datos y generar todos los tokens para un idioma dado. Luego, utilizando la función `build_vocab_from_iterator` de torchtext, se crea el vocabulario para cada idioma, incluyendo solo aquellos tokens que aparecen al menos una vez (`min_freq=1`) y añadiendo los símbolos especiales al comienzo del vocabulario:

In [ ]:
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Iterador de datos de entrenamiento
    train_iterator = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Para disminuir la cantidad de tokens de relleno, ordenas los datos según la longitud de la fuente para agrupar secuencias de longitudes similares
    sorted_dataset = sorted(train_iterator, key=lambda x: len(x[0].split()))
    # Crear el objeto Vocab de torchtext
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(sorted_dataset, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

Finalmente, se establece que el índice por defecto para los tokens desconocidos sea `UNK_IDX`. Esto evita errores cuando se intenta acceder a un token que no está presente en el vocabulario:


In [ ]:
# Si no se establece, lanza un ``RuntimeError`` cuando el token consultado no se encuentra en el vocabulario.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

#### 6. Transformación de cadenas en secuencias de índices y adición de tokens especiales

Una vez que se han construido los vocabularios, es necesario transformar las oraciones en secuencias de índices. Esto se logra en dos pasos:

1. **Tokenización y numerización:**  
   Se toma una cadena de texto, se tokeniza utilizando el tokenizador configurado y luego se convierte cada token en su índice correspondiente en el vocabulario.
2. **Añadir tokens de inicio y fin:**  
   Se añaden tokens `<bos>` y `<eos>` para marcar el inicio y el final de la secuencia. Además, en el caso del idioma de origen (alemán), se invierte la secuencia para facilitar el entrenamiento en algunos modelos (por ejemplo, en RNNs).

Para ello se definen dos funciones:  
- `tensor_transform_s` para el idioma de origen, que invierte la secuencia y añade los tokens BOS y EOS.
- `tensor_transform_t` para el idioma de destino, que solo añade los tokens BOS y EOS.

```python
def tensor_transform_s(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.flip(torch.tensor(token_ids), dims=(0,)),
                      torch.tensor([EOS_IDX])))

def tensor_transform_t(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))
```

Ejecutar estas funciones sobre las secuencias obtenidas permite obtener tensores listos para ser alimentados al modelo. Por ejemplo, se transforma una secuencia en inglés y otra en alemán:

In [ ]:
seq_en = vocab_transform['en'](token_transform['en'](english))
print(f"Cadena de texto en inglés: {english}\nSecuencia en inglés: {seq_en}")

seq_de = vocab_transform['de'](token_transform['de'](german))
print(f"Cadena de texto en alemán: {german}\nSecuencia en alemán: {seq_de}")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# función para añadir BOS/EOS, invertir la oración de origen y crear un tensor para los índices de la secuencia de entrada
def tensor_transform_s(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.flip(torch.tensor(token_ids), dims=(0,)),
                      torch.tensor([EOS_IDX])))

# función para añadir BOS/EOS y crear un tensor para los índices de la secuencia de entrada
def tensor_transform_t(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))


Posteriormente, se aplican las transformaciones definidas para añadir los tokens especiales y, en el caso del alemán, invertir la secuencia:

In [ ]:
seq_en = tensor_transform_s(seq_en)
seq_en

In [ ]:
seq_de = tensor_transform_t(seq_de)
seq_de

El resultado son tensores que contienen los índices correspondientes a los tokens de la oración, incluyendo los marcadores de inicio y fin.

#### 7. Agrupación de transformaciones: función `sequential_transforms`

Para simplificar el proceso de aplicar múltiples transformaciones en secuencia (tokenización, numerización y adición de tokens especiales), se define una función auxiliar llamada `sequential_transforms`. Esta función acepta un número variable de funciones y las aplica de forma secuencial sobre una entrada de texto y se pueden agrupar las transformaciones necesarias para cada idioma y almacenar el resultado en el diccionario `text_transform`. 

Esto permite, por ejemplo, que para el idioma de origen se apliquen de manera secuencial el tokenizador, la conversión a índices y la transformación que invierte la secuencia, mientras que para el idioma de destino se aplique la transformación sin invertir:

In [ ]:
# función auxiliar para agrupar operaciones secuenciales
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# Transformaciones de texto para los idiomas ``src`` y ``tgt`` para convertir cadenas sin procesar en índices de tensores
text_transform = {}

text_transform[SRC_LANGUAGE] = sequential_transforms(token_transform[SRC_LANGUAGE],  # Tokenización
                                            vocab_transform[SRC_LANGUAGE],         # Numerización
                                            tensor_transform_s)                    # Añadir BOS/EOS y crear tensor

text_transform[TGT_LANGUAGE] = sequential_transforms(token_transform[TGT_LANGUAGE],  # Tokenización
                                            vocab_transform[TGT_LANGUAGE],         # Numerización
                                            tensor_transform_t)                    # Añadir BOS/EOS y crear tensor

Con esto, una cadena sin procesar puede convertirse directamente en un tensor listo para ser usado en el modelo, aplicando todas las transformaciones definidas en orden.

#### 8. Procesamiento de datos en lotes: función de colación (`collate_fn`)

La función de colación es un componente clave cuando se trabaja con DataLoader en PyTorch. Su objetivo es agrupar muestras individuales en un solo lote y, en este caso, asegurarse de que todas las secuencias tengan longitudes uniformes mediante la aplicación de padding.

La función `collate_fn` definida en el código realiza lo siguiente:

1. **Inicialización de listas para almacenar secuencias de origen y destino:**  
   Se crean dos listas, `src_batch` y `tgt_batch`, que almacenarán las secuencias transformadas para cada muestra.

2. **Aplicación de transformaciones de texto:**  
   Para cada par (origen y destino) en el lote, se elimina el salto de línea al final de la cadena con `rstrip("\n")` y se aplica la transformación correspondiente definida en `text_transform`. Cada secuencia resultante se convierte en un tensor de tipo `int64`.

3. **Conversión y preparación para el padding:**  
   Se añaden las secuencias procesadas a las listas correspondientes.

4. **Aplicación de padding:**  
   Se utiliza la función `pad_sequence` de PyTorch para rellenar las secuencias de cada lote a la misma longitud, utilizando el índice `PAD_IDX` como valor de relleno. Además, se especifica `batch_first=True` para que el tensor resultante tenga la forma `[batch_size, sequence_length]`.

5. **Envío a dispositivo:**  
   Finalmente, se transfieren los tensores al dispositivo configurado (CPU o GPU) para su posterior procesamiento durante el entrenamiento.

El código de la función `collate_fn` es el siguiente:


In [ ]:
# función para agrupar muestras de datos en tensores por lotes
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_sequences = text_transform[SRC_LANGUAGE](src_sample.rstrip("\n"))
        #src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
        src_sequences = src_sequences.clone().detach().to(torch.int64)
        tgt_sequences = text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n"))
        #tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)
        tgt_sequences = tgt_sequences.clone().detach().to(torch.int64)
        src_batch.append(src_sequences)
        tgt_batch.append(tgt_sequences)

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=True)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX, batch_first=True)
    
    return src_batch.to(device), tgt_batch.to(device)

Este proceso asegura que cada lote contenga secuencias de igual longitud y listas para ser utilizadas como entrada en un modelo transformer u otro modelo de traducción.

#### 9. Configuración del DataLoader para el entrenamiento y validación

Con todas las transformaciones y funciones definidas, se procede a configurar los iteradores para el entrenamiento y la validación. Se establece un tamaño de lote (por ejemplo, 4) y se ordenan las muestras del conjunto de datos de entrenamiento según la longitud de la oración de origen. Este ordenamiento ayuda a agrupar secuencias de longitudes similares, lo que a su vez reduce la cantidad de padding necesario en cada lote:


In [ ]:
BATCH_SIZE = 4

train_iterator = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
sorted_train_iterator = sorted(train_iterator, key=lambda x: len(x[0].split()))
train_dataloader = DataLoader(sorted_train_iterator, batch_size=BATCH_SIZE, collate_fn=collate_fn, drop_last=True)

valid_iterator = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
sorted_valid_dataloader = sorted(valid_iterator, key=lambda x: len(x[0].split()))
valid_dataloader = DataLoader(sorted_valid_dataloader, batch_size=BATCH_SIZE, collate_fn=collate_fn, drop_last=True)


src, trg = next(iter(train_dataloader))
src, trg

El parámetro `drop_last=True` se utiliza para descartar el último lote si éste no alcanza el tamaño especificado, lo que puede ser útil para evitar inconsistencias en el tamaño del lote durante el entrenamiento. Se muestra cómo obtener un lote de datos del DataLoader de entrenamiento. Las variables `src` y `trg` contienen las secuencias de origen y destino, respectivamente, listas para ser procesadas por el modelo.

Estas líneas extraen el siguiente lote del iterador y lo imprimen, lo que permite verificar que el procesamiento de datos ha sido exitoso y que el formato de los tensores es el esperado.